In [107]:
from google.colab import drive
drive.mount('/content/gdrive/')

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
PATH = '/content/gdrive/My Drive/hoge/'

In [0]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import SGD, Adam
from keras.utils import np_utils
from keras.applications import VGG16

In [0]:
classes = ['apple', 'tomato', 'starawberry']
num_classes = len(classes)
IMAGE_SIZE = 224 # Specified size of VGG16

In [0]:
X_train, X_test, y_train, y_test = np.load(PATH + 'image_files.npy', allow_pickle=True)

In [0]:
# convert one-hot vector
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

In [0]:
# normalization
X_train = X_train.astype('float') / 255.0
X_test = X_test.astype('float') / 255.0

In [114]:
# include_top ; Whether to read last layer
# -> Add your own fully conected layer
vgg16_model = VGG16(weights='imagenet', include_top=False, input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
vgg16_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [115]:
# add 2 top layers
top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16_model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(num_classes, activation='softmax'))
top_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 256)               6422784   
_________________________________________________________________
dropout_8 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 771       
Total params: 6,423,555
Trainable params: 6,423,555
Non-trainable params: 0
_________________________________________________________________


In [116]:
# combine models
model = Model(inputs=vgg16_model.input, outputs=top_model(vgg16_model.output))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# Fix the weight just before the last conv layer of VGG16
for layer in model.layers[:15]:
    layer.trainable = False

In [118]:
opt = SGD(lr=1e-4, momentum=0.9)
# default : lr=0.001
# opt = Adam(lr=0.0001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])
history = model.fit(X_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
697/697 [==============================] - 8s 12ms/step - loss: 1.1889 - acc: 0.4189
Epoch 2/10
697/697 [==============================] - 7s 10ms/step - loss: 0.8108 - acc: 0.6356
Epoch 3/10
697/697 [==============================] - 7s 10ms/step - loss: 0.6790 - acc: 0.7288
Epoch 4/10
697/697 [==============================] - 7s 10ms/step - loss: 0.4960 - acc: 0.8364
Epoch 5/10
697/697 [==============================] - 7s 10ms/step - loss: 0.3951 - acc: 0.8637
Epoch 6/10
697/697 [==============================] - 7s 10ms/step - loss: 0.3027 - acc: 0.9010
Epoch 7/10
697/697 [==============================] - 7s 10ms/step - loss: 0.2511 - acc: 0.9168
Epoch 8/10
697/697 [==============================] - 7s 10ms/step - loss: 0.2097 - acc: 0.9369
Epoch 9/10
697/697 [==============================] - 7s 10ms/step - loss: 0.1812 - acc: 0.9426
Epoch 10/10
697/697 [==============================] - 7s 10ms/step - loss: 0.1633 - acc: 0.9412


In [119]:
score = model.evaluate(X_test, y_test, batch_size=32)
print('loss: {0} - acc: {1}'.format(score[0], score[1]))

233/233 [==============================] - 2s 10ms/step
loss: 0.22179728605716525 - acc: 0.9227467816275077


In [0]:
model.save(PATH + 'vgg16_transfer.h5')